# ODE solver

This notebook contains

- Euler's method

- 2nd order Runke-Kutta method

- 4th order Runge-Kutta method

$ f^{n+1} = f(x^{n+1}) = f(x^n+\Delta x) $


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import style
#style.available
style.use('seaborn-v0_8-darkgrid')

## Euler's method

First order difference approximation.

$$ \frac{f^{n+1}-f^n}{\Delta x} = \dot{f}^n +  \mathcal{O}(\Delta x) $$

$$ f^{n+1} = f^n + \Delta x\dot{f}^n +  \mathcal{O}(\Delta x^2) $$

- First order accurate

When we have multiple coupled first order differential equations, we can collect them in a vector $\vec{f}$. Applying Euler's method to this $\vec{f}$ directly applies the method on its components as well.

### Simple pendulum



In the case of a simple pendulum with angle $\theta$ and angular velocity $\omega$

$$ \vec{a} = \begin{bmatrix} \theta \\ \omega \end{bmatrix} $$

$$ \dot{\vec{a}} = \begin{bmatrix} \dot{\theta} \\ \dot{\omega} \end{bmatrix} = \begin{bmatrix} \omega \\ -\frac{g}{L}\sin(\theta) \end{bmatrix} $$

$$ \frac{a^{n+1}-a^n}{\tau} = \dot{a}^n + \mathcal{O} (\tau) $$

$$ a^{n+1} = a^n + \tau \dot{a}^n + \mathcal{O}\left(\tau^2\right) $$

$$ \begin{bmatrix} \theta \\ \omega \end{bmatrix} ^{n+1} = \begin{bmatrix} \theta \\ \omega \end{bmatrix} ^{n} + \tau \begin{bmatrix} \omega \\ -\frac{g}{L}\sin(\theta) \end{bmatrix} ^{n} + \mathcal{O}\left(\tau^2\right) $$

$$ \begin{bmatrix} \theta^{n+1} \\ \omega^{n+1} \end{bmatrix} = \begin{bmatrix} \theta ^{n} \\ \omega ^{n} \end{bmatrix} + \tau \begin{bmatrix} \omega ^{n} \\ -\frac{g}{L}\sin(\theta ^{n}) \end{bmatrix} + \mathcal{O}\left(\tau^2\right) $$

In [ ]:
class SimplePendulum:
    """ Manage and integrate a simple pendulum """

    def __init__(self, theta0, g=9.81, L=9.81, method="Euler"):
        """ We'll take theta0 in degrees and assume that the angular
        velocity is initially 0 """
        
        # Initial condition
        self.theta0 = np.radians(theta0)

        self.g = g
        self.L = L

    def rhs(self, theta, omega):
        """ Equations of motion for a pendulum
              dtheta/dt = omega
              domega/dt = - (g/L) sin theta """

        return np.array([omega, -(self.g / self.L) * np.sin(theta)])

    def integrate_euler(self, dt, tmax):
        """ Integrate the equations of motion using Euler's method """

        # Initial conditions
        t = 0.0

        t_s     = [t]
        theta_s = [self.theta0]
        omega_s = [0.0]

        while t < tmax:

            # Initial state
            theta = theta_s[-1]
            omega = omega_s[-1]
            
            # Get the RHS
            thetadot, omegadot = self.rhs(theta, omega)

            # Advance
            theta_new = theta + dt * thetadot
            omega_new = omega + dt * omegadot

            t += dt

            # Store
            t_s.append(t)
            theta_s.append(theta_new)
            omega_s.append(omega_new)

        return np.array(t_s), np.array(theta_s), np.array(omega_s)

In [ ]:
theta0  = 10.0 # Initial angle in degrees -- the class converts to radians
dt      = 0.1
tmax    = 20.0

p10 = SimplePendulum(theta0)
t_euler, theta_euler, omega_euler = p10.integrate_euler(dt, tmax)

fig, ax = plt.subplots()
ax.plot(t_euler, theta_euler, label="Euler")
ax.set_xlabel("t [s]")
ax.set_ylabel(r"$\theta(t)$");

## 2nd order Runge-Kutta method

Know the centered derivative is second order accurate, so we take a half-step in between on the form

$$ f^{n+1} = f^n + \Delta x\dot{f}^{n+\frac{1}{2}} +  \mathcal{O}(\Delta x^3) $$

- Locally third order accurate

- Globally second order accurate

Use Euler's method to predict the half-way point. Two step process looks like

$$ f^{\star} = f^n + \frac{\tau}{2}\dot{f}^n  $$

Giving the full update 

$$ f^{n+1} = f^n + \tau f^{\star} $$

In [ ]:
def integrate_RK2(state0, tau, T):

    times   = []
    history = []
    
    # Initialize time
    t = 0
    
    # Store the initial conditions
    times.append(t)
    history.append(state0)
    
    # Main timestep loop
    while t < T:
        
        # Always the newly added state
        state_old = history[-1]
        
        # Make sure that the last step does not take us past T
        if t + tau > T:
            tau = T - t

        # Get the RHS
        fdot = "insert_here".rhs(state_old)
    
        # Predict the state at the midpoint
        state_tmp = state_old + 0.5 * tau * fdot
        
        # Evaluate the RHS at the midpoint
        fdot = "insert_here".rhs(state_tmp)
        
        # Do the final update
        state_new = state_old + tau * fdot
        t += tau
        
        # Store the state
        times.append(t)
        history.append(state_new)
        
    return times, history

In [ ]:
T   = 1
tau = 0.1

state0 = "initial_conditions"

times, history = integrate_RK2(state0, tau, T)

## 4th order Runge-Kutta

General system of first order differential equations $\dot{y}=f(t, y)$ (vector)

$$ y^{n+1} = y^n + \frac{\tau}{6} (k_1 + 2k_2 + 2k_3 + k_4) $$

- $ k_1 = f(t^n, y^n) $

- $ k_2 = f(t^n + \tau/2, y^n + (\tau/2)k_1) $

- $ k_3 = f(t^n + \tau/2, y^n + (\tau/2)k_2) $

- $ k_4 = f(t^n + \tau, y^n + \tau k_3) $

Linear combination, but weighted unequally

- Globally fourth order accurate

- Locally fifth order accurate

Fixed timestep, see RK45 for adaptive timestepping

In [ ]:
def integrate_RK4(state0, tau, T):

    times   = []
    history = []
    
    # Initialize time
    t = 0
    
    # Store the initial conditions
    times.append(t)
    history.append(state0)
    
    # Main timestep loop
    while t < T:
        
        state_old = history[-1]
        
        # Make sure that the last step does not take us past T
        if t + tau > T:
            tau = T - t

        # Get the RHS
        k1 = "insert_here".rhs(state_old)
         
        state_tmp = state_old + 0.5 * tau * k1
        k2 = "insert_here".rhs(state_tmp)
        
        state_tmp = state_old + 0.5 * tau * k2
        k3 = "insert_here".rhs(state_tmp)
        
        state_tmp = state_old + tau * k3
        k4 = "insert_here".rhs(state_tmp)
        
        # Do the final update
        state_new = state_old + tau / 6.0 * (k1 + 2*k2 + 2*k3 + k4)
        t += tau
        
        # Store the state
        times.append(t)
        history.append(state_new)
        
    return times, history

In [ ]:
T   = 1
tau = 0.1

state0 = "initial_conditions"

times, history = integrate_RK4(state0, tau, T)

## SciPy

In [ ]:
from scipy import integrate

In [9]:
def rhs():
    """ Right hand side function """

In [ ]:
yvec0 = "initial_conditions"

sol = integrate.solve_ivp(rhs, (0, tmax), yvec0, method="RK45")